In [22]:
import pandas as pd

In [23]:
import numpy as np

In [24]:
from math import sqrt

In [25]:
house_data = pd.read_csv('kc_house_data.csv')
house_data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000,2,1.00,770,10000,1,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000,4,3.00,1960,5000,1,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000,3,2.00,1680,8080,1,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [26]:
train_data = pd.read_csv('kc_house_train_data.csv')

In [27]:
test_data = pd.read_csv('kc_house_test_data.csv')

Next write a function that takes a data set, a list of features (e.g. [‘sqft_living’, ‘bedrooms’]), to be used as inputs, and a name of the output (e.g. ‘price’). This function should return a features_matrix (2D array) consisting of first a column of ones followed by columns containing the values of the input features in the data set in the same order as the input list. It should also return an output_array which is an array of the values of the output in the data set (e.g. ‘price’).

In [28]:
def get_numpy_data(data, features, output):
    data['constant'] = 1 # add a constant column to a dataframe
    # prepend variable 'constant' to the features list
    features = ['constant'] + features
    # select the columns of dataframe given by the ‘features’ list into the SFrame ‘features_sframe’

    # this will convert the features_sframe into a numpy matrix with GraphLab Create >= 1.7!!
    features_matrix = data[features].as_matrix(columns=None)
    # assign the column of data_sframe associated with the target to the variable ‘output_sarray’

    # this will convert the SArray into a numpy array:
    output_array = data[output].as_matrix(columns=None) # GraphLab Create>= 1.7!!
    return(features_matrix, output_array)

If the features matrix (including a column of 1s for the constant) is stored as a 2D array (or matrix) and the regression weights are stored as a 1D array then the predicted output is just the dot product between the features matrix and the weights (with the weights on the right). Write a function ‘predict_output’ which accepts a 2D array ‘feature_matrix’ and a 1D array ‘weights’ and returns a 1D array ‘predictions’. e.g. in python:

In [29]:
def predict_outcome(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

If we have a the values of a single input feature in an array ‘feature’ and the prediction ‘errors’ (predictions - output) then the derivative of the regression cost function with respect to the weight of ‘feature’ is just twice the dot product between ‘feature’ and ‘errors’. Write a function that accepts a ‘feature’ array and ‘error’ array and returns the ‘derivative’ (a single number). e.g. in python:

In [30]:
def feature_derivative(errors, feature):
    errors = predictions - output
    derivative = 2* np.dot(errors,feature)
    return(derivative)

Now we will use our predict_output and feature_derivative to write a gradient descent function. Although we can compute the derivative for all the features simultaneously (the gradient) we will explicitly loop over the features individually for simplicity. Write a gradient descent function that does the following:

Accepts a numpy feature_matrix 2D array, a 1D output array, an array of initial weights, a step size and a convergence tolerance.
While not converged updates each feature weight by subtracting the step size times the derivative for that feature given the current weights
At each step computes the magnitude/length of the gradient (square root of the sum of squared components)
When the magnitude of the gradient is smaller than the input tolerance returns the final weight vector.

In [31]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False
    weights = np.array(initial_weights)
    while not converged:
        # compute the predictions based on feature_matrix and weights:
        predictions = np.dot(feature_matrix, weights)
        # compute the errors as predictions - output:
        errors = predictions - output
        gradient_sum_squares = 0 # initialize the gradient
        # while not converged, update each weight individually:
        for i in range(len(weights)):
            # Recall that feature_matrix[:, i] is the feature column associated with weights[i]
            # compute the derivative for weight[i]:
            derivative = 2* np.dot(errors, feature_matrix[:,i])
            # add the squared derivative to the gradient magnitude
            gradient_sum_squares += derivative**2
            # update the weight based on step size and derivative:
            weights[i] = weights[i] - step_size * derivative
        gradient_magnitude = sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)

Now we will run the regression_gradient_descent function on some actual data. In particular we will use the gradient descent to estimate the model from Week 1 using just an intercept and slope. Use the following parameters:

features: ‘sqft_living’
output: ‘price’
initial weights: -47000, 1 (intercept, sqft_living respectively)
step_size = 7e-12
tolerance = 2.5e7

In [32]:
simple_features = ['sqft_living']
my_output= 'price'
(simple_feature_matrix, output) = get_numpy_data(train_data, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

Use these parameters to estimate the slope and intercept for predicting prices based only on ‘sqft_living’.

In [33]:
simple_weights = regression_gradient_descent(simple_feature_matrix, output,initial_weights, step_size, tolerance)

Quiz Question: What is the value of the weight for sqft_living -- the second element of ‘simple_weights’ (rounded to 1 decimal place)?

In [34]:
simple_weights

array([-46999.88716555,    281.91211918])

Now build a corresponding ‘test_simple_feature_matrix’ and ‘test_output’ using test_data. Using ‘test_simple_feature_matrix’ and ‘simple_weights’ compute the predicted house prices on all the test data.

In [35]:
(test_simple_feature_matrix, test_output) = get_numpy_data(test_data,simple_features,my_output)

In [36]:
predicted_house_prices = predict_outcome(test_simple_feature_matrix, simple_weights)

Quiz Question: What is the predicted price for the 1st house in the Test data set for model 1 (round to nearest dollar)?

In [40]:
predicted_house_prices[0]

356134.44325500238

Now compute RSS on all test data for this model. Record the value and store it for later

In [45]:
RSS_model1 = np.sum((predicted_house_prices - test_output)**2)
RSS_model1

275400044902128.31

Now we will use the gradient descent to fit a model with more than 1 predictor variable (and an intercept). Use the following parameters:

In [48]:
model_features = ['sqft_living', 'sqft_living15']
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, model_features,my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

Note that sqft_living_15 is the average square feet of the nearest 15 neighbouring houses.

Run gradient descent on a model with ‘sqft_living’ and ‘sqft_living_15’ as well as an intercept with the above parameters. Save the resulting regression weights.


In [49]:
regression_weights = regression_gradient_descent(feature_matrix, output,initial_weights, step_size, tolerance)

In [50]:
regression_weights

array([ -9.99999688e+04,   2.45072603e+02,   6.52795267e+01])

Quiz Question: What is the predicted price for the 1st house in the TEST data set for model 2 (round to nearest dollar)

In [53]:
(test_feature_matrix, test_output) = get_numpy_data(test_data,model_features,my_output)

In [54]:
predicted_house_prices_model2 = predict_outcome(test_feature_matrix, regression_weights)

In [55]:
predicted_house_prices_model2[0]

366651.41162949387

What is the actual price for the 1st house in the Test data set

In [57]:
test_data['price'][0]

310000.0

Quiz Question: Which estimate was closer to the true price for the 1st house on the TEST data set, model 1 or model 2?

Now compute RSS on all test data for the second model. Record the value and store it for later.

In [58]:
RSS_model2 = np.sum((predicted_house_prices_model2 - test_output)**2)
RSS_model2

270263443629803.56

Quiz Question: Which model (1 or 2) has lowest RSS on all of the TEST data?

In [59]:
RSS_model1 > RSS_model2

True